In [87]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def sum(a: int, b: int) -> int:
    """Sum or Add two numbers."""
    
    return a + b

tools = [multiply, sum]

model = init_chat_model(model="openai:gpt-4o")
model_with_tools = model.bind_tools(tools)

tool_call = model_with_tools.invoke("what's 42 * 7?")

tool_node = ToolNode(tools)
result = tool_node.invoke({"messages": [tool_call]})
result['messages']



[ToolMessage(content='294', name='multiply', tool_call_id='call_isyCvnuwmArmRmkKuruxpEWU')]

In [29]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

from dotenv import load_dotenv

load_dotenv()

def youtube_search(query: str) -> str:
    """
    Search YouTube for videos matching the given query and return a summary of the top results.

    Args:
        query (str): The search keywords or phrase to look up on YouTube.

    Returns:
        str: A summary or list of the top YouTube video results for the query.
    """
    # Placeholder: Integrate with YouTube Data API for real results
    return f"YouTube results for: {query}"

def google_search(query: str) -> str:
    """
    Search Google for the provided query and return a summary of the most relevant results.

    Args:
        query (str): The search keywords or phrase to look up on Google.

    Returns:
        str: A summary or list of the top Google search results for the query.
    """
    # Placeholder: Integrate with Google Custom Search API for real results
    return f"Google results for: {query}"

def weather_search(location: str) -> str:
    """
    Retrieve the current weather information for a specified location.

    Args:
        location (str): The name of the city or location to get weather data for.

    Returns:
        str: A summary of the current weather conditions at the specified location.
    """
    # Placeholder: Integrate with a weather API for real results
    return f"Weather for {location}: Sunny, 25°C"

def send_email(to: str, subject: str, body: str) -> str:
    """
    Send an email to the specified recipient with the given subject and message body.

    Args:
        to (str): The recipient's email address.
        subject (str): The subject line of the email.
        body (str): The main content of the email.

    Returns:
        str: A confirmation message indicating the email was sent.
    """
    # Placeholder: Integrate with an email service for real results
    return f"Email sent to {to} with subject '{subject}'"

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

def sum(a: int, b: int) -> int:
    """Sum or Add two numbers."""
    return a + b

total_tool = [weather_search, sum, multiply, send_email, google_search, youtube_search]

tool_node = ToolNode(total_tool)

model = init_chat_model(model="openai:gpt-4o")
model_with_tools = model.bind_tools(total_tool)

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

builder = StateGraph(MessagesState)

# Define the two nodes we will cycle between
builder.add_node("call_model", call_model)
builder.add_node("tools", tool_node)

builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")

graph = builder.compile()



In [30]:
result = graph.invoke({"messages": [{"role": "user", "content": "search in google who is the prime minister of india"}]})

In [31]:
result_messages  = result.get("messages")

In [32]:
result.get("messages")

[HumanMessage(content='search in google who is the prime minister of india', additional_kwargs={}, response_metadata={}, id='e750bbec-b8e2-44b3-b5ae-53b0cf35ed52'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kqBlK8GA71qGs3oKmlKC8zmi', 'function': {'arguments': '{"query":"current Prime Minister of India"}', 'name': 'google_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 260, 'total_tokens': 278, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BXMUdp9HzLF8oLs4hhxsZ8EJIprOn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f97a416c-7e89-4617-b788-90b4f4c329ac-0', tool_calls=[{'name': 'google_search', 'ar

In [33]:
from langchain_core.messages import ToolMessage

tool_messages = [
    {"tool_name": msg.name, "content": msg.content}
    for msg in result_messages
    if isinstance(msg, ToolMessage)
]

# Extract tool names
tool_name = [msg.name for msg in result_messages if isinstance(msg, ToolMessage)]

print(tool_name)

['google_search']


In [34]:
result.get("messages")

[HumanMessage(content='search in google who is the prime minister of india', additional_kwargs={}, response_metadata={}, id='e750bbec-b8e2-44b3-b5ae-53b0cf35ed52'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kqBlK8GA71qGs3oKmlKC8zmi', 'function': {'arguments': '{"query":"current Prime Minister of India"}', 'name': 'google_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 260, 'total_tokens': 278, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BXMUdp9HzLF8oLs4hhxsZ8EJIprOn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f97a416c-7e89-4617-b788-90b4f4c329ac-0', tool_calls=[{'name': 'google_search', 'ar

In [124]:
result = graph.invoke({"messages": [{"role": "user", "content": "who is the prime minister of india"}]})

In [125]:
result["messages"][-1].content

'As of October 2023, the Prime Minister of India is Narendra Modi.'